In [1]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import pandas as pd
import numpy as np


In [2]:
pdf_path='reliance.pdf'
pdf_path

'reliance.pdf'

In [3]:
images=convert_from_path(pdf_path)

In [4]:
for i, image in enumerate(images):
    # Save the image (optional)
    #image.save(f'page_{i+1}.png', 'PNG')

    # Use Tesseract to extract text from the image
    pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'
    text = pytesseract.image_to_string(image)

    # Print or process the extracted text
    print(text)

Q

Reliance

Industries Limited

July 19, 2024

BSE Limited National Stock Exchange of India Limited
Phiroze Jeejeebhoy Towers, Exchange Plaza, Plot No. C/1, G Block,
Dalal Street, Bandra-Kurla Complex,

Mumbai 400 001 Bandra (East), Mumbai 400 051

Scrip Code: 500325 Trading Symbol: RELIANCE

Dear Sirs,

Sub: Presentation made to analysts on the Unaudited Financial Results (Consolidated
and Standalone) for the quarter ended June 30, 2024

Pursuant to Regulation 30(6) of the Securities and Exchange Board of India (Listing Obligations
and Disclosure Requirements) Regulations, 2015, the presentation on the Unaudited Financial
Results (Consolidated and Standalone) for the quarter ended June 30, 2024, made today to the
analysts, is attached and also available on the website of the Company at
https://www.ril.com/investors/financial-reporting.

The presentation concluded at 8:51 p.m. (IST).
This is for information and record.
Thanking you

Yours faithfully,
For Reliance Industries Limited
SA

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import re
import numpy as np
from langchain_openai import OpenAIEmbeddings,ChatOpenAI

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
content_url="https://whyzest.com/course-details-llm-masterclass/"

In [6]:
resp=requests.get(content_url)
text=BeautifulSoup(resp.content).text
text

'\n\n\n\n\n\nCourse Details: LLM Masterclass – Whyzest\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWhyzest\n\n\n\n\n\n\nHomeBlogContactAbout\n\n\n\n\n\n\n\n\n\n\n\nCourse Details: LLM Masterclass\n\n\nFoundational Principles for Real-World Problem Solving\n\nGo to Course\n\nGain Deep AI Insights from a Leading Industry Expert\nCourse Overview\nDive into the world of Large Language Models (LLMs) with our meticulously crafted masterclass designed for serious AI/ML professionals. This course covers foundational concepts, practical insights and teaches first-principles approach to building LLM applications, equipping you to systematically solve hard real-world problems using generative AI.\nKey Value Propositions\n\nDeep Knowledge Course material distilled from hundreds of recent landmark papers and open-source codebases, viewed through the lens of an experienced AI practitioner.\nFirst-Princip

In [7]:
text="\n\n".join( re.split(r"\n{3,}",text))
text

'\n\nCourse Details: LLM Masterclass – Whyzest\n\nWhyzest\n\nHomeBlogContactAbout\n\nCourse Details: LLM Masterclass\n\nFoundational Principles for Real-World Problem Solving\n\nGo to Course\n\nGain Deep AI Insights from a Leading Industry Expert\nCourse Overview\nDive into the world of Large Language Models (LLMs) with our meticulously crafted masterclass designed for serious AI/ML professionals. This course covers foundational concepts, practical insights and teaches first-principles approach to building LLM applications, equipping you to systematically solve hard real-world problems using generative AI.\nKey Value Propositions\n\nDeep Knowledge Course material distilled from hundreds of recent landmark papers and open-source codebases, viewed through the lens of an experienced AI practitioner.\nFirst-Principles Approach Primary focus on core concepts and mental model development for enabling first-principle problem solving, minimizing dependency on pre-created recipes or prompts.\nI

In [8]:
'''
RecursiveCharacterTextSplitter is used to split text into chunks based on character count

chunk_size=1000: This specifies that each chunk of text should be up to 1000 characters long.
chunk_overlap=200: This specifies that there should be an overlap of 200 characters between consecutive chunks. 
This overlap can help in maintaining context between chunks and overlap is only done when necessary based on the context
'''

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
chunks=text_splitter.split_text(text)

In [9]:
print(chunks[0])

Course Details: LLM Masterclass – Whyzest

Whyzest

HomeBlogContactAbout

Course Details: LLM Masterclass

Foundational Principles for Real-World Problem Solving

Go to Course


In [10]:
#Creating A Vector Store-Used to store document embeddings and gives a method to query those embeddings by using cosine similarity

class NumpyVectorStore:
    def __init__(self,embedder=None):
        self.index=None
        self.docs=[]

        if embedder is None:
            embedder=OpenAIEmbeddings()

        self.embedder=embedder

    def add_documents(self,chunks):
        self.docs=chunks
        embeddings=self.embedder.embed_documents(chunks)#Uses the embedder to generate embeddings for the provided document chunks.
        if self.index:#Checks if self.index aldready contains embeddings
            self.index=np.concatenate((self.index,embeddings),axis=0)#If not empty,concatenate new embeddings to old ones
        else:
            self.index=np.array(embeddings)#Convert embeddings to a numpy array and store in self.index

    def query(self,q,top_k=5):#This method is used to query the vector store with a query string q and return the top top_k results
        q_embedding=self.embedder.embed_query(q)
        cosines=np.dot(self.index,q_embedding)#Computes the cosine similarities between the query embedding and all document embeddings in-- 
        #--self.index.
        ordered=np.argsort(-cosines)#Sorts the indices of the cosine similarities in descending order (highest similarity first)
        return [(self.docs[i],cosines[i]) for i in ordered[:top_k]]#Returns a list of tuples containing the top top_k document chunks and--
        #--their corresponding cosine similarity scores

In [11]:
import openai
embedder=OpenAIEmbeddings()#Key redacted for privacy

vs=NumpyVectorStore(embedder)
vs.add_documents(chunks)

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt_template=("You are an assistant for question-answering tasks.Use the following pieces"
                 "of retrieved context to answer the question.If you don't know the answer, "
                 "just say that you don't know.Use the three sentences maximum and keep the answer concise.\n"
                 "Question: {questions} \n"
                 "Context: {context} \n"
                 "Answer:")

                 

In [13]:
print(prompt_template)

You are an assistant for question-answering tasks.Use the following piecesof retrieved context to answer the question.If you don't know the answer, just say that you don't know.Use the three sentences maximum and keep the answer concise.
Question: {questions} 
Context: {context} 
Answer:


In [14]:
rag_chain=PromptTemplate.from_template(prompt_template) | ChatOpenAI(model="gpt-3.5-turbo",temperature=0) | StrOutputParser()#Key redacted for privacy


In [15]:
def answer(question):
    relevant_chunks=vs.query(question)
    context="######### CONTEXT ######### \n" + "\n\n***\n".join([chunk for chunk, score in relevant_chunks]) + "###############"
    ans=rag_chain.invoke({"context":context, "questions":question})#Give the context that is defined in the previous line and give the question
    return ans

In [16]:
query="What are we gonna learn about AI agents?"
print(answer(query))

You are going to learn about definitions, types, and applications of AI agents, as well as their role in enhancing AI capabilities in LLMs.


In [17]:
query="What are some of the concepts used?"
print(answer(query))

Some of the concepts used include understanding embeddings, few-shot prompting, and LLM reasoning capabilities.


In [18]:
query="What will we learn in Module 5?"
print(answer(query))

In Module 5, we will learn how to get started with the LangChain framework and build high-level APIs on basic string in string out API.


In [19]:
relevant=vs.query("What will we learn about LLM agents")
for chunk,score in relevant:
    print("Score:",score)
    print(chunk)
    print("#########################")

Score: 0.8386280845706783
Course Syllabus
Week 1
Module 1 – Introduction to LLMs
Overview: Learners will gain an understanding of the evolution of language models, particularly how LLMs work and their strengths and limitations.
#########################
Score: 0.8359123621057452
Module 2 – Theoretical Foundations of LLMs
Overview: Delve into the underlying principles and mechanisms of LLMs, focusing on how they work and attain their capabilities through pre-training, instruction-tuning and RLHF stages.
#########################
Score: 0.8252193964336583
Part 2:

Recap of key learnings from the course.
Discussion on future trends in the field of LLMs.
Next steps for continued learning and exploration in advanced LLM topics.

Course Schedule

Duration
6 weeks (12 sessions)

Session Dates
6th July to 11th August (Saturdays and Sundays)

Time
10:30 am to 12:30 pm (IST)

Assignment Discussions + Q&A Sessions
Wednesdays, 8:00 pm to 9:30 pm (IST)

Format
Virtual Live Sessions
################

In [20]:
relevant=vs.query("What will we learn in Module 5?")
for chunk,score in relevant:
    print("Score:",score)
    print(chunk)
    print("#########################")

Score: 0.8250120941566037
Course Objectives
#########################
Score: 0.8139173637322646
Part 2:

Learn techniques for processing unstructured data and converting it into structured information.
Understand how to process raw text, tables, images, etc., for better indexing and retrieval in RAG applications.

Week 5
Module 9 – Introduction to Agents and Their Capabilities
Overview: Introduction to agents in LLMs, their roles, and how to incorporate memory and planning for enhanced capabilities.
#########################
Score: 0.8109622447660143
Week 3
Module 5 – Getting Started with LangChain Framework
Overview: Get started with the LangChain framework, intuitively understand how they build high-level APIs on basic string in string out API of foundational models so that you can confidently use it for developing LLM applications.
#########################
Score: 0.8102087117874847
Part 2:

Dive into a detailed examination of the LangChain API.
Understand the internal workings of t

In [21]:
tata_text='''Tata Electronics' Rs 27,000-crore chip assembly plant in Assam will create 27,000 jobs and is expected to become operational in 2025, Tata Sons Chairman N Chandrasekaran said on Saturday.

Speaking on the occasion of chip plant's Bhumi Pujan ceremony, Chandrasekaran said that the company has already employed 1,000 people from Assam and as the facility expands it will bring entire semiconductor ecosystem companies.At its capacity it will employ 27,000 people, 15,000 direct jobs and additional 12,000 indirect jobs. We want to move fast. We are trying to accelerate the building of this factory. We hope sometime in 2025 we will be able to complete this facility and quickly start operations,Chandrasekaran said.

The Tata semiconductor assembly and test facility at Jagiroad is India's first indigenous semiconductor assembly and test facility.

The plant is expected to generate over 27,000 direct and indirect jobs in the region and the plant is to be operationalised by mid-2025, it will give immense boost to hi-tech manufacturing in India. The new plant is strategically located with access to abundant water and green power and proximity to current semiconductor packaging & test hubs in countries like Taiwan, Malaysia, Vietnam, and Singapore.

A 3-D model of the semiconductor assembly and test facility was unveiled on the occasion by the chief minister.

Look forward to being in Jagiroad shortly and initialise work on the semiconductor facility by performing the Bhumi pujan of the plant, Sarma had earlier posted on X.

The Chief Minister's Office (CMO) in another post on X said, Today is a historic day for Assam as HCM Dr@himantabiswa will perform the Bhumi pujan of the Tata Semiconductor Assembly and test facility at Jagiroad.

Once operational, it will transform the industrial landscape of Assam and herald a new era of growth, the CMO said.Prime Minister Narendra Modi had virtually laid the foundation stone of the facility on March 13 this year.

Tata Group is proud to spearhead the advanced electronics ecosystem in India under the visionary leadership of Hon. Prime Minister Shri Narendra Modi. The innovation driven industry will propel Assam in the global electronics global supply chain ecosystem. Tata Group has strong ties with Assam and 60,000 employees of the Group are working here in various industries, said  Chairman Tata Sons, N Chandrasekaran.

He also conveyed best wishes from Ratan Tata to the residents of Assam on this momentous occasion. The Tata Sons Chairman thanked Assam Chief Minister Himanta Biswa Sarma, Union minister Ashwani Vaishnav for helping speed up the project, which he said will provide job opportunities to 27,000 and will also open immense avenues for creating a future ready and knowledge base electronics value chain.

The greenfield project of Tata Electronics at Jagiroad will involve an investment of Rs 27,000 crore and will generate employment opportunities for more than 30,000 people.

The first phase of the facility is expected to be operational by mid-2025.

The company will build the facility focusing on three key platform technologies – wire bond, flip chip, and a differentiated offering called integrated systems packaging (ISP), with plans to expand the roadmap to advanced packaging technologies in the future.

Semiconductor assembly and testing is a critical part of the value chain where wafers manufactured by semiconductor fabs are assembled or packaged and then tested before they are finally used in the desired product.

The proposed facility will serve the growing global demands across key market segments like AI, industrial, and consumer electronics, the company said.

The project will provide an immense boost to industrialisation in North-East India. It is envisioned under the Centre's semiconductor policy being driven by the India Semiconductor Mission and the Assam government's electronics policy.'''

In [22]:
tata_text


"Tata Electronics' Rs 27,000-crore chip assembly plant in Assam will create 27,000 jobs and is expected to become operational in 2025, Tata Sons Chairman N Chandrasekaran said on Saturday.\n\nSpeaking on the occasion of chip plant's Bhumi Pujan ceremony, Chandrasekaran said that the company has already employed 1,000 people from Assam and as the facility expands it will bring entire semiconductor ecosystem companies.At its capacity it will employ 27,000 people, 15,000 direct jobs and additional 12,000 indirect jobs. We want to move fast. We are trying to accelerate the building of this factory. We hope sometime in 2025 we will be able to complete this facility and quickly start operations,Chandrasekaran said.\n\nThe Tata semiconductor assembly and test facility at Jagiroad is India's first indigenous semiconductor assembly and test facility.\n\nThe plant is expected to generate over 27,000 direct and indirect jobs in the region and the plant is to be operationalised by mid-2025, it wil

In [23]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks2=text_splitter.split_text(text)

In [24]:
vs=NumpyVectorStore(embedder)
vs.add_documents(chunks2)

In [25]:
query="Tell me when the first plant will be operational"
print(answer(query))

I don't have information on when the first plant will be operational.


In [26]:
query="Tell me everything Chandrashekan said about that plant"
print(answer(query))

I don't have information about what Chandrashekan said about that plant.


In [27]:
query="How many jobs will be created beacuse of it"
print(answer(query))

The number of jobs that will be created because of the course is not specified in the provided context.


In [28]:
query="What is the use of the plant, why is it significant"
print(answer(query))

The plant is used for agentic workflow patterns and memory integration in LLM applications. It is significant for enhancing planning capabilities and context retention in problem-solving tasks.


In [29]:
mazagon_text2='''
Shares of Mazagon Dock Shipbuilders soared 7.5 percent in trade to hit a fresh record high of Rs 4,271.20 on June 26 after the company became the latest to join the list of companies to be granted the 'Navratna' status by the Department of Public Enterprises. With this, the company became the 18th public sector undertaking (PSU) to be awarded the status.

With this award, the company joined the likes of Engineers India, CONCOR, Bharat Electronics, Hindustan Aeronautics, RCF, NALCO, NMDC, RVNL, IRCON and IREDA, all of which hold the 'Navratna' status.

With the Navratna status, Mazagon Dock gets the authority to invest up to Rs 1,000 crore without requiring approval from the central government. Additionally, it can invest up to 30 percent of its net worth annually, provided it does not exceed Rs 1,000 crore.

Navratna companies also have the flexibility to form joint ventures, establish alliances, and set up subsidiaries abroad.
To qualify as a Navratna company, a PSU must first achieve Miniratna status. It must then maintain a net profit of Rs 5,000 crore annually for three consecutive years, along with either an average annual turnover exceeding Rs 25,000 crore or an average annual net worth surpassing Rs 15,000 crore over the same period. For FY24, Mazagon Dock reported an annual turnover of Rs 9,466 crore.

Meanwhile, the stock surged over 4 percent in the previous session as well, driven by news of progressing talks to acquire more Kalvari-class (Scorpene) submarines for the Navy. The anticipated Rs 35,000-crore deal aims to bolster India's submarine fleet significantly.

Sources reported by ET revealed that the three new submarines in discussion will be larger and incorporate advanced electronics, enhancing their operational capabilities and endurance compared to previous models.

Given that Mazagon Dock Shipbuilders  is a government-owned PSU operating under the Ministry of Defence, it serves as the designated shipyard responsible for constructing naval vessels and submarines for the Indian Navy. Hence, the Rs 35,000-crore deal to expand the Indian Navy's submarine fleet stand to directly benefit Mazagon Dock.

Aside from this, the stock has also been on a roll in recent months, buoyed by the government's push on indigenisation, rising capex for the sector and growing export opportunities. Flying high on these growth levers, the stock has already delivered more than two-fold returns in the past three months.

Net Sales at Rs 3,103.65 crore in March 2024 up 49.32% from Rs. 2,078.59 crore in March 2023.

Quarterly Net Profit at Rs. 662.97 crore in March 2024 up 103.25% from Rs. 326.19 crore in March 2023.

EBITDA stands at Rs. 872.84 crore in March 2024 up 106.9% from Rs. 421.86 crore in March 2023.

Mazagon Dock EPS has increased to Rs. 32.87 in March 2024 from Rs. 16.17 in March 2023.

Mazagon Dock shares closed at 3,199.75 on June 12, 2024 (NSE) and has given 54.30% returns over the last 6 months and 206.05% over the last 12 months
After taking a beating in the past two sessions due to post-election volatility, shares of public sector undertaking (PSU) companies staged a strong comeback on June 6, surging 4-14 percent in trade.
June-
Shares of PSU companies, seen as the major beneficiaries of policy continuity with the return of the BJP to power, saw an intense selloff in the past two sessions after the Modi-led party failed to achieve a majority on its own.


Sharp profit booking amid concerns of a lag in policy continuity as the BJP needs to take support from its allies to form the government for a third time weighed heavily on these counters. As a result, most of these stocks hit their lower circuits in the past two sessions.

Follow our live blog for all the market action

Nonetheless, with the BJP set to return to power, albeit with the help of its coalition allies, several brokerages believe its policy measures, including its capex plans are unlikely to be halted.

Related stories
BHEL Standalone June 2024 Net Sales at Rs 5,484.92 crore, up 9.62% Y-o-Y  BHEL Standalone June 2024 Net Sales at Rs 5,484.92 crore, up 9.62% Y-o-Y 
Coal India: Decent show in Q1 FY25; levers in place for long-term gain  Coal India: Decent show in Q1 FY25; levers in place for long-term gain 
BHEL Q1 results: Net loss widens to Rs 211 crore, revenue rises 10%  BHEL Q1 results: Net loss widens to Rs 211 crore, revenue rises 10% 
CLSA also wrote in a note that its channel checks suggest an ambitious 100-day agenda being formulated, which is expected to drive the incoming NDA government into immediate action. The brokerage anticipates the government to kickstart work with large-scale orders in the infrastructure and defence sectors, signalling that the capex thrust will continue under Modi 3.0 as well.

These developments, along with bargain hunting revived sentiment for PSU stocks, triggering a rally in today's sessions.

Mazagon Dock Shipbuilders led gains among PSU stocks as it soared over 14 percent, followed by BHEL, which was up 13.5 percent. Other prominent gainers in the PSU pack were Ircon International, CONCOR, Cochin Shipyard, Hindustan Aeronautics, HUDCO, and REC which zoomed 9-10 percent.

Sharp gains of 4-14 percent across all constituents also lifted the BSE PSU index nearly 6 percent higher in trade.

August-

The stock market is abuzz with speculation on potential fiscal measures and their impacts ahead of the Union Budget 2024 presentation on July 23.

Defence stocks such as Bharat Electronics Ltd (BEL), Hindustan Aeronautics Ltd (HAL), Bharat Dynamics Ltd (BDL), Cochin Shipyard Ltd, and Data Patterns India Ltd, are in focus on Monday morning as the Union Budget 2024, which is scheduled for Tuesday, July 23, is expected to see higher allocation for the sector.

related stories
Ambuja Cement To Invest Rs 1,600 Crore in Bihar to Set Up Cement Grinding Unit
Ambuja Cement To Invest Rs 1,600 Crore in Bihar to Set Up Cement Grind...
JK Tyre Q1 Results: Net Profit Rises 37% to Rs 211 Crore, Total Income Falls to Rs 3,655 Crore
JK Tyre Q1 Results: Net Profit Rises 37% to Rs 211 Crore, Total Income...
SBI Q1 Results: Net Profit Up 0.9% to Rs 17,035.2 Crore, Board Approves Rs 25,000 Crore Fundraising
SBI Q1 Results: Net Profit Up 0.9% to Rs 17,035.2 Crore, Board Approve...
Akums Drugs IPO Allotment Finalised: Check GMP, Listing Date
Akums Drugs IPO Allotment Finalised: Check GMP, Listing Date...
Ambuja Cement To Invest Rs 1,600 Crore in Bihar to Set Up Cement Grinding Unit
Ambuja Cement To Invest Rs 1,600 Crore in Bihar to Set Up Cement Grind...
JK Tyre Q1 Results: Net Profit Rises 37% to Rs 211 Crore, Total Income Falls to Rs 3,655 Crore
JK Tyre Q1 Results: Net Profit Rises 37% to Rs 211 Crore, Total Income...
SBI Q1 Results: Net Profit Up 0.9% to Rs 17,035.2 Crore, Board Approves Rs 25,000 Crore Fundraising
SBI Q1 Results: Net Profit Up 0.9% to Rs 17,035.2 Crore, Board Approve...
Akums Drugs IPO Allotment Finalised: Check GMP, Listing Date
Akums Drugs IPO Allotment Finalised: Check GMP, Listing Date...
Ambuja Cement To Invest Rs 1,600 Crore in Bihar to Set Up Cement Grinding Unit
Ambuja Cement To Invest Rs 1,600 Crore in Bihar to Set Up Cement Grind...
JK Tyre Q1 Results: Net Profit Rises 37% to Rs 211 Crore, Total Income Falls to Rs 3,655 Crore
JK Tyre Q1 Results: Net Profit Rises 37% to Rs 211 Crore, Total Income...
SBI Q1 Results: Net Profit Up 0.9% to Rs 17,035.2 Crore, Board Approves Rs 25,000 Crore Fundraising
SBI Q1 Results: Net Profit Up 0.9% to Rs 17,035.2 Crore, Board Approve...
Akums Drugs IPO Allotment Finalised: Check GMP, Listing Date
Akums Drugs IPO Allotment Finalised: Check GMP, Listing Date...

To recall, Finance Minister Nirmala Sitharaman announced a budgetary allocation of Rs 1.72 lakh crore towards FY25 defence capital expenditure in her interim Budget 2024, which was 20.33 per cent higher than the actual expenditure of FY23 and 9.40 per cent more than the revised allocation of FY24.

Multibagger Solar Industries , Astra Microwave and Azad Engineering are the top picks of ICICI Securities in the Defence Sector. Solar Industries, Astra Microwave, Azad Engineering share price have risen 147-217 per cent in last one year giving Multibagger returns to investors.


We continue to remain positive on the sector, as the order pipeline for most of the companies remains healthy and managements have guided for 15–25 per cent revenue growth while maintaining the margins for FY25, said ICICI Securities in its report.

ICICI Securities also has a BUY rating on Dynamatic Technologies share Price with a target price of Rs 10,250. However, they await the award of the Pinaka order, which they believe is in advanced stages; Solar Industries share price is also the likely beneficiary. Also, they await the orders for QRSAM and MRSAM (Quick reaction and medium range Surface-to-Air Missiles ) which are likely to benefit Bharat Electronics, Bharat Dynamics and and Astra Microwave share price.

The ordering activity has been catching pace in last one month post the completion of Lok Sabha Elections.

Key developments in the month of June’2024 included the first Tejas Mk-1A aircraft is expected to be delivered in Jul’24.while Hindustan Aeronautics ltd also has received a Request for proposal from the Ministry of Defence on the procurement of 156 LCH Prachand helicopters. Hindustan Aeronautics Ltd has won orders worth Rs 1170 Crore from Cochin shipyard.

You May Like
Brigade™ Insignia. 3, 4 & 5 BHK Starting 2.90 Cr*
Brigade Group
  by TaboolaSponsored Links
Garden Reach Shipbuilders & Engineers Ltd has won orders worth Rs 1910 Crore order while Bharat Electronics has won orders worth Rs 3170 Crore as per ICICI Securities data. Cochin shipyard also has won orders worth Rs 1100 Crore while Solar industries has developed three new explosives.

The target price of Solar Industries share price at Rs 13250 by ICICI Securities indicates more than 13 per cent upside for the stock from current market price of Rs 11710. For Astra Microwave share price , target of ₹935 and Azad Engineering share price target of Rs 1750 are 2-4 per cent higher than current levels at which these share prices are trading and hence any corrections may be good opportunity to Buy.

Prabhudas Lilladher expects higher allocations towards aircraft, engines, and vehicles. Ministry of Defence will also continue its thrust on exports, having announced a Rs 50,000 crore defence export target by FY29, it said.


This brokerage said India’s defence story remains intact due to unrelenting focus on enhancing domestic production capabilities, improving technology of arms and armaments and push for exports.

It sees increased order inflow in segments like fighter aircraft, helicopters, armoured vehicles, missiles and naval platforms. Each platform will require new electronics such as advanced avionics, radars, fire control systems, as well as various ancillary equipment, it said.

This brokerage said a sustained focus would keep defence stocks such as HAL, BEML, BEL, Mazagon Dock Shipbuilders, Bharat Dynamics, GRSE, Data Patterns and Cochin Shipyard buzzing.

Ahead of the Budget 2024, MOFSL initiated coverage on Zen Technologies, calling it a niche player in the defense simulator-based training market.

“The company has also forayed into the counter-drone market. The addressable markets for simulators and counter-drones in India are estimated at Rs 14,000 crore and Rs 12,000 crore, respectively, over the next five years. The company commands a significant market share in both of these segments, with just 2-3 players in the defense simulator market and 5-6 players in the counter-drone market,” it said.


Fisdom Research said stocks such as BEL, HAL and DDL to be on investor radar.

'''

In [30]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=400)
chunks3=text_splitter.split_text(mazagon_text2)

In [31]:
vs=NumpyVectorStore(embedder)
vs.add_documents(chunks3)

In [32]:
query="Tell me how Mazagon is doing these days"
print(answer(query))

Mazagon Dock Shipbuilders has been performing well, with a significant increase in net sales, net profit, EBITDA, and EPS. The stock has delivered strong returns in the past three and six months, and recently hit a record high after being granted 'Navratna' status.


In [33]:
query="Compare to other defense companies and tell me whats going on recently"
print(answer(query))

Recent developments in the defense sector include increased order inflow in various segments like fighter aircraft, helicopters, armoured vehicles, missiles, and naval platforms. Companies like HAL, BEML, BEL, and others are expected to benefit from sustained focus and positive investor sentiment. Additionally, the defense sector is anticipating higher allocations in the upcoming budget, with companies like Solar Industries, Astra Microwave, and Azad Engineering being top picks for investment.


In [34]:
query="Any bad news in the sector?"
print(answer(query))

There is no bad news in the sector, as the stock market is abuzz with speculation on potential fiscal measures and their impacts ahead of the Union Budget 2024 presentation. Defense stocks are in focus as the Union Budget 2024 is expected to see higher allocation for the sector.


In [35]:
query="How's it doing in July compared to August"
print(answer(query))

The BSE PSU index saw sharp gains of 4-14 percent in July compared to August. Defense stocks were in focus in July due to the Union Budget 2024 presentation. The BJP's return to power boosted sentiment for PSU stocks in July.


In [36]:
query="Any corrections in the price lately"
print(answer(query))

There have been sharp gains of 4-14 percent across all constituents, lifting the BSE PSU index nearly 6 percent higher in trade. The stock market is abuzz with speculation on potential fiscal measures and their impacts ahead of the Union Budget 2024 presentation on July 23. Defence stocks are in focus as the Union Budget 2024 is expected to see higher allocation for the sector.


In [37]:
reliance_text='''
Shri Mukesh D. Ambani
Chairman and Managing Director, Reliance Industries
Adoption of concepts like circularity
in our chemical products also brings
us closer to a low carbon economy.
We have partnered with accomplished
innovator entities globally, with
whom we share the common goal
of establishing a sustainable world
economy. We are leveraging their
expertise to derive maximum value for
our new energy initiatives. We recognise
the urgency of addressing the issues
emanating from climate change. We
believe our new energy initiatives will
contribute to the global effort of limiting
the rise in average temperatures.
It is important for us to ensure the
wellbeing of our employees and
communities. We lay great emphasis on
the health, safety and development of
all our employees, customers, business
partners and local communities.
Human rights considerations have
been at the core of Reliance’s DNA
since inception. The principles of
human rights are enshrined in our
internal codes of conduct as well as our
business partner codes of conduct. Our
Reliance Foundation works round the
clock driving initiatives in education,
healthcare and social development
that have impacted millions of lives
over the years. Abiding by our principle
of inclusive growth, our teams across
businesses work at a grass root
level to identify the needs of every
stakeholder involved and commit to
securing their interests and maintaining
their satisfaction.
Managing large and diverse
businesses is not possible without a
robust governance structure. We, as
a company, have always strived
to maintain the highest levels of
governance in our management.
We emphasise the importance of
Board oversight, given the fact that
the company’s diverse and evolving
operations impact essential sectors
of the Indian economy. The Board
Committees at Reliance have clearly
defined roles and mandates. We are
committed to adhering to the best
practices and to adapting to emerging
standards of governance.
Pursuing the path of good governance,
we constituted the ESG committee
to ensure our continued alignment to
best ESG practices and compliance
with applicable ESG frameworks. The
committee will advise the board and
other committees on environmental,
social and governance aspects involved
in the conduct of business activities.
We have entered an era of
transformation – transformation to a
greener and a more inclusive economy.
Our businesses, our people and our
practices are evolving and adapting to
the new age with a renewed zest. Our
newest growth engine, our new energy
business is poised to become a pioneer
and lead the way towards just global
energy transition.
Creation of an Independent
Financial Services Business
My dear friends, as you are aware
that this year we have demerged our
financial services arm into a separate
listed entity named “Jio Financial
Services Limited” (JFS). I thank you
for your wholehearted support to
this initiative.
Today’s India is young and
entrepreneurial, adopting digital
finance at an unprecedented pace. The
digital revolution has penetrated every
corner of the nation through Jandhan
Accounts, digital payments, usage of
smart phones and low cost data. The
growth opportunities presented by
financial services are remarkable and
provide a strong directional support to
the economy.
As various financial services are governed
by different regulatory frameworks,
we believe, an independent financial
services entity will allow us to access
the opportunities available in the
Indian market.
Jio Financial Services Limited along
with its subsidiaries will leverage the
technological capabilities of Reliance
and digitally deliver financial services,
democratising access to financial
services offerings for Indian citizens.
Jio Financial Services aims to provide
simple, affordable and innovative digital
first solutions.
Jio Financial Services Limited is
positioned uniquely to capture the growth
opportunities in financial services sector
and play a crucial role in transforming the
landscape of digital finance in India.
Financial Performance
Summary for the Year
There was continued volatility in the
global markets amidst disruptions in
trade flows and economic uncertainties.
Despite the headwinds, Reliance
was able to deliver another year
of remarkable performance both
on operating and financial fronts.
Consolidated EBITDA was at `1,53,920
crore which is a robust 24.4% growth as
compared to last year. Consolidated net
profit also grew at 11.3% to `73,670 crore.
The impressive growth was well rounded
and supported by all our business
segments. Consumer businesses
continued their growth trajectory with
a rapid expansion of retail store footprint
and strong subscriber acquisition
in Jio. O2C business delivered a record
performance amidst a challenging
business environment.
A switch to cleaner energy
sources is key to our
decarbonisation strategy.
We are making significant
strides in establishing a
world class solar energy
value chain. We are also
progressing on building a
green hydrogen ecosystem.
Our goal is to provide
affordable green hydrogen
as a viable alternative to
traditional fuels.
Our goal is to become Net Carbon Zero
by 2035. Accordingly, the development
of giga factories at the Dhirubhai
Ambani Green Energy Giga Complex
at Jamnagar is progressing rapidly.
Considering the collective potential
of our five giga factories, we are well
on track for establishing a world class,
self-sufficient green energy ecosystem.
The transition from traditional fossil
fuels to renewable energy sources
is going to be a crucial milestone in
our history. We, as a company, have
a proven record of creating value
through transformational changes, be
it the digital revolution through Jio,
transforming consumer experience
and the retail landscape in India or
the integration of our refining and
petrochemicals businesses into the oil
to chemicals value chain.
CONSOLIDATED EBITDA
H1,53,920CRORE
CONSOLIDATED NET PROFIT
H73,670CRORE
Dear and esteemed
fellow shareholders,
We, at Reliance, have a
deep-rooted belief that
sustainability is an integral
part of enterprise growth.
The tremendous stress that
mankind has exerted on
natural resources, since the
advent of the Industrial
Revolution, has brought
us to a stage where we
need to collectively ramp
up our efforts to combat
climate change.
The Paris Agreement of
2015 entered into at the
21st Conference of Parties
(COP 21) brought the world
together in this pursuit.
Our company’s ESG
ambitions are aligned
with this global quest. On
a personal note, it is a
privilege for me to serve on
the Advisory Committee of
COP 28 UAE, and contribute
to this ongoing effort


Oil and gas segment also registered
a strong performance with sustained
gas production from KG D6 block.
Elevated inflation levels led to an
aggressive rate hike by central banks
through the financial year. Our
prudent financial framework helped
optimise the returns and minimise
the risks while providing adequate
liquidity to our businesses. Our strong
cash flows from operations have ably
supported growth capex requirements.
Over the past 2 years, 98% of the
company’s capex has been funded by
internally generated cash profits. This
has helped us maintain a strong and
conservative balance sheet and retain
superior investment grade ratings.
Robust Operational
Performance
Reliance touches the lives of
millions of consumers in India and
worldwide through its energy and
consumer businesses. Our teams
across businesses work dedicatedly
to achieve operational excellence and
deliver quality performance.
Premier League on Jio Cinema. Jio
Platforms powered the technology
behind streaming these events. The
new home broadband back up plan is
the latest addition to the bouquet of
broadband services and is expected to
grow the broadband market further.
Jio retained its preeminent position
in the Indian telecom space with
439.3 million subscribers as on
March 2023, with a net addition of
29.2 million subscribers during the
year. The strong growth in the digital
services business is aptly reflected in
its financial performance for the year.
With an increase of 24.9% from the
previous year, digital services EBITDA
for FY 2022-23 stood at `50,286 crore.
Jio commits itself to continually
innovate and launch newer
technologies and products and
expand the market for digital services.
Jio True 5G stack is one such feature
loaded indigenous technology,
offering superior performance. It is
secure, cost effective and also easy to
install. This technology is fully cloud
native, software defined, digitally
managed and supports advanced
features like Quantum Security.
Jio AirFiber is also an innovative
solution, offering clutter free and
high speed connectivity to users with
the capability of supporting multiple
devices simultaneously without
compromising on the quality of
connectivity. JioDive is a technology
that helps users to explore the virtual
reality universe through a smartphone
based headset. JioDive runs on the
JioImmerse application which offers
users exclusive VR experiences in
gaming, learning, entertainment
and wellness, all accessible through
their smartphone.
I believe Jio’s digital offerings have
established it as a strong market
leader in India’s digital services
ecosystem. The scale of Jio’s
operation and advantages of deep
penetration help Jio grow stronger
with the launch of each new offering.
Jio’s vision is to make the highest level
of connectivity available to the masses
at affordable prices at all times.
Retail
Reliance Retail further consolidated
its position as the largest retailer
in India and the only Indian retailer
to feature in the top 100 retailers
globally. FY 2022-23 witnessed
Reliance Retail cross the 1 billion
transactions mark.
The retail business expanded its
store network with over 3,300 new
stores added during the financial year,
taking the store tally to 18,040 stores
Pan-India. Our stores now cover
65.6 million sq ft of store area which
is a more than 50% growth of retail
space as compared to the previous
year. Footfalls increased by 50%
from last year to 780 million. The
registered customer base also grew
to 249 million, establishing Reliance
Retail as the preferred shopping venue
of Indian citizen across formats.
During the year, Reliance Retail
entered into the beauty segment by
launching the “Tira” digital platform
and opening its flagship store in
Mumbai. The business also expanded
its product basket through acquisitions
and partnerships at both local and
global levels like Metro, Campa Cola,
GAP, Pret A Manger, Lotus, Sosyo,
Maliban, Toffeeman to name a few.
Reliance Retails runs an integrated
network of physical stores, digital
commerce and new commerce
initiatives. Through digital brands like
Ajio and Netmeds, digital commerce
is scaling up at a rapid pace and is
contributing to the strong growth of
retail segment. The launch of JioMart
on WhatsApp is a truly novel initiative
which combines online shopping
experience with the simplicity of
instant chat services. Growth in
new commerce business has been
fast paced with rapid expansion
of its merchant partner network.
Currently, more than 3 million
merchants have partnered with our
new commerce platforms.
Retail segments EBITDA stood at
`17,974 crore which reflects a strong
44.7% increase from the previous
year. The EBITDA growth was well
supported by all verticals right from
grocery to fashion and lifestyle.
Reliance Retail is an indispensable part
of the Indian household’s shopping
experience not only in metro regions
but also in tier 2 and tier 3 cities.
Reliance Retail envisions to expand
its offerings and scale of operations
to make best in class products
available to the Indian consumers at
affordable prices.
During the year our teams across sites
worked towards finding sustainable
solutions. The first ever green
hydrogen production was achieved
with firing of torrefied biomass in
gasifiers. Biomass firing in Coal Fired
Boilers (CFBs) at petrochemical
sites increased to >11% of total feed
in line with our decarbonisation
efforts. Ethanol blending in motor
spirit (EBMS) facility was also
commissioned. We also successfully
commissioned the demo plant for
our patented Continuous Catalytic
Pyrolysis oil technology.
Our O2C business is transforming.
From switch to renewable sources
of energy and newer energy
technologies to promoting the
concept of circular economy especially
for our petrochemical products,
sustainable business techniques are
rapidly gaining momentum.
MJ field at the KG D6 Block
commenced production post testing
and commissioning activities. At
peak production of ~30 MMSCMD,
KG D6 block will contribute ~30%
of the domestic gas production.
Operationalization of the KG D6 MJ gas
field is a crucial step towards enhancing
India’s energy security.
Conclusion
Dear fellow shareholders, Reliance,
as a company has grown multifold by
delivering value to the citizens, the
country and global community. Our
products are ingrained in the lives of the
citizens and are an integral part of their
daily routines. Our various businesses
have always identified the needs of the
society and worked towards devising
and making available timely and
affordable solutions.
The current needs of the society are
sustainable solutions which can tackle
climate change through affordable
green energy and inclusive growth. Our
initiatives in New Energy businesses and
our consumer businesses are aimed at
meeting societal needs and aspirations.
Using the right technology, deploying
the right talent and through empowering
our young leaders, I am confident that
we can make the world a better place to
live in for future generations.
I would like to express my sincere
appreciation to the Board of Directors
for their continued guidance. I would
like to thank our teams across
businesses, for their dedicated work
and commitment to excellence. I would
also like to extend my gratitude to all
our stakeholders for their support and
unwavering faith in Reliance.
With best wishes,
Mukesh D. Ambani
Chairman and Managing Director
August 4, 2023
Digital Services
Digital services continued its
strong performance through the
strengthening of existing offerings
and introducing newer services in
its portfolio. During the year, Jio
launched the True 5G services in India.
By the end of the financial year, Jio’s
5G coverage expanded to 2,300+
cities and towns, demonstrating
an impressive pace of network
installation. The rollout of FTTH
services also gained momentum
during the year.
Millions of viewers enjoyed the live
action of the FIFA World Cup, the
first ever Women’s Premier League
and the 16th edition of the Indian
Oil to Chemicals
Despite facing major operational
challenges given the highly volatile
global scenarios, our O2C business
posted highest ever earnings
excluding the impact of SAED.
Revenues increased on account
of improved price realisation for
transportation fuel, tracking higher
average oil prices for the year. O2C
business EBITDA stood at `62,075
crore with a healthy 17.7% increase
as compared to last year despite
considering the SAED levy of
`6,648 crore. EBITDA growth was
driven by feedstock flexibility, sharp
improvements in fuel cracks and
advantageous ethane cracking.
During the year, our refinery
processed 13 new variants of crude
oil, broadening the feedstock sources.
Demand for transportation fuels
was strong for the year reflecting
the opening up of economies post
pandemic restrictions. Demand for
downstream chemical products
remained subdued globally, but was
strong in the domestic markets.
Oil
'''

In [38]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=5000,chunk_overlap=1000)
chunks4=text_splitter.split_text(reliance_text)

In [39]:
vs=NumpyVectorStore(embedder)
vs.add_documents(chunks4)

In [40]:
def answer2(question,max_length,num_chunks):
    relevant_chunks=vs.query(question,top_k=num_chunks)
    context="######### CONTEXT ######### \n" + "\n\n***\n".join([chunk for chunk, score in relevant_chunks]) + "###############"
    ans=rag_chain.invoke({"context":context, "questions":question, "max_length":max_length})#Give the context that is defined in the previous line and give the question
    return ans

In [41]:
query="Tell me what all ambani said about growth and their expansion plans"
print(answer(query))

Mukesh D. Ambani emphasized the importance of sustainable growth and inclusive development in their expansion plans. They are committed to transitioning to cleaner energy sources and becoming Net Carbon Zero by 2035. Their new energy initiatives, including green hydrogen production and solar energy value chain, are key components of their growth strategy.


In [42]:
query="Tell me what he said about retail, how is went this year, their plans going forward"
print(answer(query))

Reliance Retail consolidated its position as the largest retailer in India, crossing 1 billion transactions. They expanded their store network with over 3,300 new stores and increased footfalls by 50%. Their EBITDA for the retail segment increased by 44.7% from the previous year.


In [43]:
query="Tell me everything that is run by Reliance Retail"
print(answer2(query, max_length=200,num_chunks=10))

Reliance Retail is run by Reliance Industries and is the largest retailer in India, with over 18,000 stores across the country. They have also expanded into the beauty segment and have partnerships with various brands.


In [44]:
query="talk about the financial performannce of the company"
print(answer2(query, max_length=200,num_chunks=10))

The company showed a robust financial performance with a 24.4% growth in EBITDA and an 11.3% increase in net profit. The growth was supported by all business segments, including consumer businesses and O2C business. The company aims to become Net Carbon Zero by 2035 through its decarbonisation strategy and focus on cleaner energy sources.


In [45]:
query="talk about the operational performance and highlight the new products added"
print(answer2(query, max_length=200,num_chunks=10))

The operational performance of Reliance has been robust, with a focus on sustainable solutions and new energy technologies. The addition of new products like Jio True 5G services and JioMart on WhatsApp has contributed to the growth of digital services. The company's financial performance has shown strong growth, with consolidated EBITDA at `1,53,920 crore and consolidated net profit at `73,670 crore.


In [46]:
query="talk about the operational performance and highlight the new products added"
print(answer2(query, max_length=200,num_chunks=20))

The operational performance of Reliance has been robust, with a focus on sustainable solutions and new energy technologies. The addition of new products like Jio True 5G services and JioMart on WhatsApp has contributed to the growth of digital services. The company's financial performance has shown strong growth, with consolidated EBITDA increasing by 24.4% and net profit by 11.3%.


In [47]:
query="talk about the operational performance and highlight the new products added"
print(answer2(query, max_length=500,num_chunks=20))

The operational performance of Reliance has been robust, with a focus on sustainable solutions and new energy technologies. The addition of new products like Jio True 5G services and JioMart on WhatsApp has contributed to the growth of digital services. The company's financial performance has shown strong growth, with consolidated EBITDA at `1,53,920 crore and consolidated net profit at `73,670 crore.


In [52]:
!pip install langchain_community

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
import os